<a href="https://colab.research.google.com/github/BeccaOY/BeccaOY/blob/main/Customer_Subscription_Status_Prediction_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

BUSINESS OBJECTIVE:
- Train a model to predict customers who will unsubscribe in July 2018

BUSINESS HYPOTHESES: 
Customers who unsubcribe are likely...
- Received too many emails
- Acquired recently
- Not as engaged 
	- less store visit
	- less email open & click
	- spend less
	- shop in fewer product areas
	- no dillards' credit card
- Live further from store

READ IN DATA USING PANDAS

In [188]:
import pandas as pd




In [189]:
sub_status = pd.read_csv('customer_subscription_status.csv')
email_activity = pd.read_csv('customer_email_campaign_agg.txt')
customer_info = pd.read_csv('customer.txt')
store_location= pd.read_csv('store_location.txt')
purchase_info = pd.read_csv('item.txt')


 TESTING FIRST HYPOTHESIS, TOO MANY EMAILS


In [190]:
#view data
email_activity.head()


,customer_id,total_email_sent_count,total_email_open_count,total_email_click_count,product_email_sent_count,product_email_open_count,product_email_click_count,sale_email_sent_count,sale_email_open_count,sale_email_click_count,last_email_open_date,last_email_click_date
0,225440671,55,16,0,41,12,0,3,1,0,7/1/2018,?
1,226090744,29,3,1,24,1,0,2,1,1,6/29/2018,6/28/2018
2,180891722,24,0,0,17,0,0,3,0,0,?,?
3,166847570,22,0,0,15,0,0,2,0,0,?,?
4,195739838,2,1,0,1,0,0,1,1,0,6/27/2018,?


In [191]:
sub_status.head()

,customer_id,unsubscribed_yn
0,223824501,N
1,225074975,N
2,225962452,N
3,224503675,N
4,204405913,N


In [192]:
#perform feature engineering by recoding y, n to 0,1
def recode(series):
    if series == 'N':
      return 1
    if series == 'Y':
      return 0
sub_status['unsubscribed_yn'] = sub_status['unsubscribed_yn'].apply(recode)

In [193]:
#remove null values
sub_status.dropna()


,customer_id,unsubscribed_yn
0,223824501,1
1,225074975,1
2,225962452,1
3,224503675,1
4,204405913,1
...,...,...
24995,226433167,0
24996,207298191,0
24997,226565446,0
24998,224727461,0


In [194]:
sub_status.head()

,customer_id,unsubscribed_yn
0,223824501,1
1,225074975,1
2,225962452,1
3,224503675,1
4,204405913,1


In [195]:
x_activity = email_activity[['total_email_sent_count', 'total_email_open_count']]
target = sub_status['unsubscribed_yn']

In [196]:
#Scale total 
x = x_activity.iloc[:22100, ]
x

,total_email_sent_count,total_email_open_count
0,55,16
1,29,3
2,24,0
3,22,0
4,2,1
...,...,...
22095,26,25
22096,28,0
22097,15,0
22098,29,0


In [197]:
y = target.iloc[:22100,]
y.unique()

array([1, 0])

SPLIT DATA

In [198]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .33)

BUILD THE MODEL

In [199]:
from sklearn.linear_model import LogisticRegression
#set regularization to generaliza and avoid overfitting
reg = .01
model = LogisticRegression(C=1/reg,solver='liblinear').fit(x_train,y_train)
print(model)

LogisticRegression(C=100.0, solver='liblinear')


EVALUATE THE MODEL

In [200]:
predictions_too_many_emails = model.predict(x_test)
print('Actual labels: ', pd.array(y_test))
print('Predictions: ', predictions_too_many_emails)

Actual labels:  <PandasArray>
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
 ...
 0, 1, 1, 1, 0, 1, 1, 1, 1, 1]
Length: 7293, dtype: int64
Predictions:  [1 1 1 ... 1 1 1]


CHECK ACCURACY


In [201]:
from sklearn.metrics import accuracy_score


In [202]:
accuracy = accuracy_score(y_test, predictions_too_many_emails)
print('Accuracy: ', accuracy)

Accuracy:  0.9036061977238448


The Model was able to predict subscription status with 90% accuracy given  number of emails

TESTING SECOND HYPOTHESIS:
ACQUIRED RECENTLY


TESTING FOURTH HYPOTHESIS: LIVE FURTHER FROM STORE

In [215]:
customer_location =customer_info['household_distance_to_nearest_store']
customer_location.count() 

25000

In [217]:
#Remove unknown values
import numpy as np
customer_location_new = customer_location.replace("?", np.nan)
customer_location_new.dropna(inplace=True)
customer_location_new.count()

17473

In [226]:
#x = customer_location_new.iloc[ :17400]
x = customer_location
x = x.loc[:22000]
x

0              ?
1              ?
2              ?
3              ?
4          4.708
          ...   
21996          ?
21997     75.386
21998    158.298
21999          ?
22000     15.097
Name: household_distance_to_nearest_store, Length: 22001, dtype: object

In [229]:
y = y.loc[:22000]

SPLIT THE DATA

In [230]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x ,y , test_size = .33, )
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)


(14740,)
(7261,)
(14740,)
(7261,)


BUILD MODEL

In [231]:
from numpy import array
x_array = array(x_train)
x_train = x_array.reshape(-1,1)

In [232]:
from sklearn.linear_model import LogisticRegression
#set regularization to generaliza and avoid overfitting
reg = .01
model = LogisticRegression(C=1/reg,solver = 'liblinear').fit(x_train,y_train)
print(model)

ValueError: ignored